# Tensor Network Renormalization Group for the 2D Ising model

In [6]:
import functools
import matplotlib.pyplot as plt
import numpy

Define the basic functions for the Ising model

Define Kronecker Delta

In [7]:
def kronecker(i: int, j: int) -> int:
    return int(i == j)

In [8]:
kronecker(1,1), kronecker(0,1)

(1, 0)

Define the weight functions of Ising model

$Z_2$ has two representations, $k=0$ and $k=1$. The weight on an edge is $\cosh(\beta)$ for $k=0$, $\sinh(\beta)$ for $k=1$.

In [9]:
def weight(beta: float, k: int) -> float:
    return (1 - k) * numpy.cosh(beta) + k * numpy.sinh(beta)

In [10]:
weight(0, 0), weight(0, 1), weight(1, 1)

(1.0, 0.0, 1.1752011936438014)

Define the initial tensor

The tensor is 4-valent, each edge carrying a representation of $Z_2$, and a square root of an edge weight (for $\beta$ and $k$). On the vertex, we have a Kronecker Delta of the sum of the 4 representations meeting at the vertex modulo 2; if this sum is 1, it vanishes.

In [11]:
def tensor(beta: float, m: int = 2):
    r = numpy.arange(0, m) # give us the [0, 1] range which we'll use for each axis
    iv, jv, kv, lv = numpy.meshgrid(*(4*[r])) # compute vectorized "index" tensors - numerical
    # trick for making this recipe a few orders of magnitude faster
    raw = numpy.sqrt(weight(beta, iv) * weight(beta, jv) * weight(beta, kv) * weight(beta, lv))
    delta_mask = 1 - ((iv + jv + kv + lv) % m)
    return delta_mask * raw

In [14]:
t = tensor(beta=1.0)
t

array([[[[2.38109785, 0.        ],
         [0.        , 1.8134302 ]],

        [[0.        , 1.8134302 ],
         [1.8134302 , 0.        ]]],


       [[[0.        , 1.8134302 ],
         [1.8134302 , 0.        ]],

        [[1.8134302 , 0.        ],
         [0.        , 1.38109785]]]])

In [13]:
len(t[:,0,0,0])

2

Splitting tensors for the singular value decomposition

This will be similar to the introductory Julia File. We pair the indices of the tensor in the way we want to split it, and define a matrix. On this matrix we will apply a singular value decomposition, truncate here at 2 singular values, and define new 3-valent tensors from it.

Translating back just means splitting the combined index again into two separate ones.

The function should return two 3-valent tensors and a list of singular values.

Normalize the singular values with respect to the largest one (so the largest singular value is always one).

We need to the same again, but this time for the other splitting.

To get the new effective tensor, we now have to contract the fine degrees of freedom.

To compute the flow of renormalized tensors, we have to iterate these procedures a few times. We write this into a main function.

We have to specify the number of iterations of the algorithm, then define the initial tensor. We can pass $\beta$ as a parameter of the main function.

Then we do a for loop, in which we do the first splitting, second splitting and then contract the new three valent tensor. Then we repeat.

For each iteration we should store the singular values computed in the singular value decomposition. We will afterwards plot them to distinguish the phases of the Ising model and finding the phase transition.